In [1]:
import pandas as pd
import numpy as np
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import callback_context

In [2]:
import os 
os.chdir = ("D:\\Vignesh\\Personal\\Data Analyst\\Recent Projects\\Financial Dashboard\\Data")

In [3]:
df = pd.read_csv('Data\\data.csv')

In [4]:
cls_df = pd.pivot_table(df,index = ['Direction','State/UT','Year','Contact'],columns = 'Type', values= "Amount",aggfunc = 'sum').reset_index()
cls_df['profit'] = cls_df['Sales']-cls_df['Purchases']-cls_df['Expense']


In [5]:
pl = []
Abs_Amount = []
for i in cls_df['profit']:
    if i > 0:
        pl.append('Profit')
        Abs_Amount.append(abs(i))
    else:
        pl.append('Loss')
        Abs_Amount.append(abs(i))
cls_df['profit/Loss'] = pl
cls_df['Abs_profit'] = Abs_Amount

cls_df

Type,Direction,State/UT,Year,Contact,Expense,Purchases,Sales,profit,profit/Loss,Abs_profit
0,East,Manipur,2020,Person Incharge : C,3028369.496,3.858194e+06,9.692646e+06,2806082.984,Profit,2806082.984
1,East,Mizoram,2020,Person Incharge : D,1885207.746,2.672893e+06,3.328934e+06,-1229167.087,Loss,1229167.087
2,East,Nagaland,2020,Person Incharge : A,2445708.381,3.086555e+06,7.754117e+06,2221853.608,Profit,2221853.608
3,North,Jammu and Kashmir,2020,Person Incharge : B,4021776.510,5.702172e+06,1.331573e+07,3591786.220,Profit,3591786.220
4,North,Ladakh,2020,Person Incharge : A,5027220.638,7.127714e+06,1.664467e+07,4489732.769,Profit,4489732.769
5,North,Punjab,2020,Person Incharge : B,3016332.383,4.276629e+06,5.326294e+06,-1966667.339,Loss,1966667.339
6,South,Karnataka,2020,Person Incharge : C,6434842.413,9.123474e+06,2.130517e+07,5746857.955,Profit,5746857.955
7,South,Kerala,2020,Person Incharge : D,4826131.804,6.842606e+06,8.522070e+06,-3146667.723,Loss,3146667.723
8,South,Tamil Nadu,2020,Person Incharge : D,8043553.013,1.140434e+07,2.663147e+07,7183572.447,Profit,7183572.447
9,West,Gujarat,2020,Person Incharge : A,1927356.653,2.411371e+06,6.057904e+06,1719176.147,Profit,1719176.147


In [6]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )
server = app.server

In [9]:
app.layout = dbc.Container([

    dbc.Row(
        dbc.Col(html.H1("Financial Dashboard",
                        className='text-center text-primary mb-4'),
                width=12)
    ),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='treemap', figure=px.treemap(
                data_frame=cls_df,
                path=['Year', 'Direction', 'State/UT', ],
                values='Abs_profit',
                color='profit/Loss',
                template='plotly_dark',
                color_discrete_map={
                    'Profit': 'Green',
                    'Loss': 'crimson'},
                custom_data=['State/UT']),
                      )], xs=12, sm=12, md=12, lg=12, xl=12),

        dbc.Row([
            dbc.Col([
                dcc.Graph(id='starburst', figure={},
                          )], xs=12, sm=12, md=12, lg=6, xl=6,className='w-50 p-3'),
            dbc.Col([
                dcc.Graph(id='line', figure={}
                          )], xs=12, sm=12, md=12, lg=6, xl=6,className='w-50 p-3'),
        ]),
    ])
])


@app.callback(
    Output(component_id='starburst', component_property='figure'),
    Input(component_id='treemap', component_property='hoverData'),
    Input(component_id='treemap', component_property='clickData'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def update_starburst(hover_data, click_data):
    if click_data is None and hover_data is None:
        entry = "2020"
    elif click_data is None:
        entry = hover_data['points'][0]['id']
    else:
        entry = click_data['points'][0]['id']

    entry_list = entry.split('/')
    
    for i in entry_list:
        if i in list(df['State/UT']):
            ddf = df.loc[df['State/UT'] == i]
            tx = f'Summarised chart for the {i} State/UT'
        elif i in ["North", "South", "East", "West"]:
            ddf = df.loc[df['Direction'] == i]
            tx = f'Summarised chart for the {i} Region'
        else:
            ddf = df
            tx = 'Summarised chart for the period 2020'

    figure = px.sunburst(ddf, path=['Type', 'Account'], values='Amount', template='plotly_dark',
                         title=tx, custom_data=['State/UT', 'Account'])
    return figure

@app.callback(
    Output(component_id='line', component_property='figure'),
    Input(component_id='starburst', component_property='hoverData'),
    Input(component_id='starburst', component_property='clickData'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def line(hover_data, click_data):
    if click_data is None and hover_data is None:
        entry = ["2020"]
    elif click_data is None:
        entry = hover_data['points'][0]['customdata']
    else:
        entry = click_data['points'][0]['customdata']

    for i in entry:
        if i in list(df['Account']):
            ddf = df.loc[(df['State/UT'] == i) | (df['Account'] == i)]  # Updated condition
            tx = f'Monthly breakup chart for the Account {i}'  # Updated title
        else:
            ddf = df
            tx = 'Monthly breakup chart for the period 2020'

    figure = px.bar(ddf, x="Month", y='Amount', template='plotly_dark', color="Account",
                     title=tx)
    return figure

In [8]:
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
entry = 2020 
entry_list = entry.split('/')

In [ ]:
entry_list

## Stacked Bar Chart

<b> Bar chart is stacked through assigning color, and text

In [ ]:
fig =px.bar(cls_df,
            x='Direction',
            y='profit',
            color='profit/Loss',
            text = 'State/UT',
            template = 'plotly_dark',
            color_discrete_map ={
                'Profit':'Green',
                'Loss':'crimson'
            }
           )
fig.update_layout(title_text ='Segment wise performance',title_x = 0.5)
fig.show()

## Sunburst Chart

Most Interactive chart

Path defines the herarchy to drill down

In [ ]:
state = 'Tamil Nadu'

In [ ]:
df.columns

In [ ]:
fig = px.sunburst(df,
                  path=['Type','Account',],
                  values='Amount',
                  template = 'plotly_dark',
                  color_discrete_map ={
                'Sales':'Green',
                'Expense':'crimson',
                  'Purchases':'Grey'}
                  
                         
)
fig.show()

In [ ]:
cls_df.columns